# Simple Search Engine
---

## Scrapping

Using the crawler script, scrape the Marvel wiki website.

```
python ./crawler
```

***Note***


The data from this action will be saved under the wiki folder.

## Dependencies

Import all dependencies and create needed folders

***Note***

The following packages needs to be present on host machine.

- requests
- nltk
- beautifulsoup4

In [1]:
import os
import re
import glob
import json
import string
import collections

import nltk
import requests
from bs4 import BeautifulSoup

# download NLTK dependencies
nltk.download('punkt')
nltk.download('stopwords')

# create a new folder (corpus) to hold all cleaned text content from scrapped pages
os.makedirs(os.path.join(os.getcwd(), "corpus"), exist_ok=True)
# create a new folder (tables) to hold needed index tables
os.makedirs(os.path.join(os.getcwd(), "table"), exist_ok=True)

[nltk_data] Downloading package punkt to /Users/nana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Text Processing

Using BeautifulSoup and regex:

- discard all redundant parts of the HTML pages (eg. scripts, headers, footers etc)
- strip all tags from the HTML pages so only text content remains
- remove all punctuations
- cast all text to lowercase

***HINT***

Below block takes about 1 minute to execute (due to size of scrapped docs).

In [2]:
# Extract only the relevant text content from the html file into a text file
def parser(html_path):
    corpus_path = os.path.join(os.getcwd(),'corpus', os.path.basename(html_path).rsplit('.', 1)[0] + '.txt')

    with open(html_path, 'r') as html_file:
        with open(corpus_path, 'w') as corpus_file:
            soup = BeautifulSoup(html_file.read(), 'html.parser')

            # remove content table
            soup.find(id='toc').decompose()
            # remove superscripts
            for sup in soup.find_all('sup'):
                sup.decompose()
            # remove footnotes
            while(soup.find('h2', text='See Also').find_next_sibling()):
                soup.find('h2', text='See Also').find_next_sibling().decompose()
            soup.find('h2', text='See Also').decompose()

            # extract text content from current updated html content
            corpus_text = soup.select('.mw-parser-output')[0].get_text("\n", strip=True)
            # remove all punctuations from text content
            corpus_text = corpus_text.translate(str.maketrans('', '', string.punctuation))
            # remove all non alphanumeric characters
            corpus_text = re.sub(r'[^A-Za-z0-9]+', ' ', corpus_text)
            # cast all letters to lowercase
            corpus_text = corpus_text.lower()

            corpus_file.write(corpus_text)

# get path reference to each scrapped wiki page and call parser function on each
for html_path in glob.iglob(os.path.join(os.getcwd(), "wiki/*.html")):
    parser(html_path)
    print(f'\r Cleaning --> {html_path}', end='')

print('\r' + 'All documents saved under corpus folder.\nSystem may print all text on a single line without word wrapping.')

All documents saved under corpus folder.
System may print all text on a single line without word wrapping.


## Index Building

Using NLTK:
- tokenise each cleaned up page 
- remove stop words
- build document table
- build vocabulary table
- build postings table

***Note***

Cast arrays to set data structures for speed

#### Tokenisation helper

In [3]:
# list of file path reference to all corpus (cleaned HTML pages)
corpus_paths = [i for i in glob.iglob(os.path.join(os.getcwd(), 'corpus/*.txt'))]

# tokenise and stem a corpus
# parameter1 - string (corpus)
def tokeniser(corpus):
    tokens = []
    ps = nltk.stem.PorterStemmer()
    sw = set(nltk.corpus.stopwords.words('english'))
    
    for term in set(nltk.word_tokenize(corpus)):
        stemmed_term = ps.stem(term)
        if stemmed_term not in sw:
            tokens.append(stemmed_term)
    
    return tokens

#### Document table builder

In [5]:
# build and save docid table to file
# return - the docids table as a dictionary
def docids_table():
    print('\r Document table building ...', end='')

    uid = 1
    dic = {}
    sorted_dic = collections.OrderedDict()

    # map url to id
    for corpus_path in corpus_paths:
        url = os.path.basename(corpus_path).rsplit('.', 1)[0]
        dic[url] = uid
        uid += 1

    # sort the dictionary alphabetically
    for i in sorted(dic.keys()):
        sorted_dic[i] = dic[i]


    # save dictionary as json file
    with open(os.path.join(os.getcwd(), 'table', 'docids.json'), 'w') as f:
        f.write(json.dumps(sorted_dic))

    return dic

#### Vocabulary table builder

In [6]:
# build and save vocabulary table to file
def vocabulary_table():
    print('\r Vocabulary table building ...', end='')

    uid = 1
    dic = {}
    sorted_dic = collections.OrderedDict()

    # map term to id
    for corpus_path in corpus_paths:
        with open(corpus_path, 'r') as f:
            terms = tokeniser(f.read())
            for term in terms:
                if term not in dic:
                    dic[term] = uid
                    uid += 1

    # sort the dictionary alphabetically
    for i in sorted(dic.keys()):
        sorted_dic[i] = dic[i]

    # save dictionary as json file
    with open(os.path.join(os.getcwd(), 'table', 'vocabulary.json'), 'w') as f:
        f.write(json.dumps(sorted_dic))
    
    return dic

#### Postings table builder

In [7]:
# build and save postings table to file
# parameter1 - docids table as a dictionary
# parameter2 - vocabulary table as a dictionary
def postings_table(docids_dic, vocabulary_dic):
    print('\r Postings table building ...', end='')

    dic = {}
    sorted_dic = collections.OrderedDict()
    
    # map term to (term id, document ids,  document frequency)
    for corpus_path in corpus_paths:
        url = os.path.basename(corpus_path).rsplit('.', 1)[0]
        with open(corpus_path, 'r') as f:
            terms = tokeniser(f.read())
            for term in terms:
                if term not in dic:
                    dic[term] = { 'voc_id': vocabulary_dic[term], 'doc_ids': [ docids_dic[url] ], 'doc_freqs': [1] }
                else:
                    if docids_dic[url] not in dic[term]['doc_ids']:
                        dic[term]['doc_freqs'].append(1)
                    else:
                        for i,j in enumerate(dic[term]['doc_ids']):
                            if j == docids_dic[url]:
                                dic[term]['doc_freqs'][i] = dic[term]['doc_freqs'][i] + 1
                                
                    doc_ids = [ *dic[term]['doc_ids'], *([docids_dic[url]] if docids_dic[url] not in dic[term]['doc_ids'] else []) ]
                    dic[term] = { 'voc_id': vocabulary_dic[term], 'doc_ids': doc_ids, 'doc_freqs': dic[term]['doc_freqs'] }
               
    # sort the dictionary alphabetically
    for i in sorted(dic.keys()):
        sorted_dic[i] = dic[i]

    # save dictionary as json file
    with open(os.path.join(os.getcwd(), 'table', 'postings.json'), 'w') as f:
        f.write(json.dumps(sorted_dic))

    return dic

#### Trigger tables build

In [8]:
docids_dic = docids_table()
vocabulary_dic = vocabulary_table()
postings_dic = postings_table(docids_dic, vocabulary_dic)
print('\r All tables built and saved under table directory.\n A dictionary copy of each table has be set to memory for referencing.', end='')

 All tables built and saved under table directory.
 A dictionary copy of each table has be set to memory for referencing.

## Boolean Retrieval

#### Single term extraction

Enter a single term query to get the document list which contains query.

The document list is sorted from most relevant to least relevant using term frequency.

***Note***

Next code block requires user input.

Use the example ```name``` to get all documents since each Avenger has a name.

***Testing***

Copy and paste any single word from a particular avenger's page as query. And the function should return that avengers document url in results.

In [9]:
def single_term(query):
    dic = {}
    try:
        # remove punctuations, cast to lowercase and tokenise query
        tokens = tokeniser(query.lower().translate(str.maketrans('', '', string.punctuation)))
        # get match from postings table
        post = postings_dic.get(tokens[0])
    
        for idx, post_doc_id in enumerate(post['doc_ids']):
            for url, doc_id in docids_dic.items():
                if post_doc_id == doc_id:
                    dic[url] = post['doc_freqs'][idx]

        # sort the dictionary by frequency and print results
        ranked_result = [k for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)]

        if len(ranked_result):
            print(*ranked_result, sep='\n')
            # uncomment to get a print out of the sorted rankings map
            # print({k: v for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)})
        else:
            raise Exception()
        
    except Exception:
        print('No matches found!')
    
query = input('Enter a single term query (eg. Name):').split()
if len(query) == 1:
    single_term(query[0].strip())
elif len(query) < 1:
    print('No term entered!')
elif len(query) > 1:
    print('More than one term entered!')

Samuel_Wilson_(Earth-616)
Reed_Richards_(Earth-616)
Brian_Braddock_(Earth-616)
Kamala_Khan_(Earth-616)
Jennifer_Walters_(Earth-616)
Peter_Parker_(Earth-616)
Hercules_Panhellenios_(Earth-616)
Anthony_Stark_(Earth-616)
Wanda_Maximoff_(Earth-616)
Miles_Morales_(Earth-1610)
Steven_Rogers_(Earth-616)
Jonathan_Hart_(Earth-616)
Nadia_Van_Dyne_(Earth-616)
Wendell_Vaughn_(Earth-616)
James_Rhodes_(Earth-616)
Janet_Van_Dyne_(Earth-616)
Robert_Baldwin_(Earth-616)
Clinton_Barton_(Earth-616)
Thor_Odinson_(Earth-616)
Pietro_Maximoff_(Earth-616)
Roberto_Reyes_(Earth-616)
Jane_Foster_(Earth-616)
Gilgamesh_(Earth-616)
Carol_Danvers_(Earth-616)
T%27Challa_(Earth-616)
Angelica_Jones_(Earth-616)
Bruce_Banner_(Earth-616)
Namor_McKenzie_(Earth-616)
Anthony_Druid_(Earth-616)
James_Howlett_(Earth-616)
Venom_(Symbiote)_(Earth-616)
Roberto_Da_Costa_(Earth-616)
Eugene_Thompson_(Earth-616)
Samuel_Alexander_(Earth-616)
Shang-Chi_(Earth-616)
Otto_Octavius_(Earth-616)
Scott_Lang_(Earth-616)
Stephen_Strange_(Earth-616

#### Two term retrieval

Enter a two terms query to get the document list which contains query.

Each term should be a **single** world.

Look at the complex term function for multi term queries.

The document list is sorted from most relevant to least relevant using term frequency.

***Note***

Next code block requires user input.

Use the example ```name AND history``` to get all documents since each Avenger has a name and history.

***Testing***

Copy and paste any two words from a particular avenger's page as query. And the function should return that avengers document url in results.

In [10]:
def two_term(query1, query2):
    dic = {}
    get_doc_url = lambda lookup_id: [url for url, doc_id in docids_dic.items() if lookup_id == doc_id ][0]

    try:
        # remove punctuations, cast to lowercase and tokenise query
        tokens1 = tokeniser(query1.lower().translate(str.maketrans('', '', string.punctuation)))
        tokens2 = tokeniser(query2.lower().translate(str.maketrans('', '', string.punctuation)))
        # get match from postings table
        post1 = postings_dic.get(tokens1[0])
        post2 = postings_dic.get(tokens2[0])

        min_post = post1 if len(post1['doc_ids']) < len(post2['doc_ids']) else post2
        max_post = post1 if len(post1['doc_ids']) > len(post2['doc_ids']) else post2
        for i in range(len(min_post['doc_ids'])):
            for j in range(len(max_post['doc_ids'])):
                if min_post['doc_ids'][i] == max_post['doc_ids'][j]:
                    url = get_doc_url(min_post['doc_ids'][i])
                    dic[url] = min_post['doc_freqs'][i] + max_post['doc_freqs'][j]
                    break
        
        # sort the dictionary by frequency and print results
        ranked_result = [k for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)]

        if len(ranked_result):
            print(*ranked_result, sep='\n')
            # uncomment to get a print out of the sorted rankings map
            # print({k: v for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)})
        else:
            raise Exception()

    except Exception:
        print('No matches found!')
    
query = input('Enter a two term query with \'AND\' delimeter (eg. History AND Powers):').split('AND')
if len(query) == 2:
    two_term(query[0].strip(), query[1].strip())
elif len(query) < 2:
    print('Missing \'AND\' delimeter!')
elif len(query) > 2:
    print('More than two terms entered!')

Steven_Rogers_(Earth-616)
Thor_Odinson_(Earth-616)
Bruce_Banner_(Earth-616)
James_Howlett_(Earth-616)
Marcus_Milton_(Earth-13034)


#### Complex queries

Enter any number term query to get the document list which contains query.

The document list is sorted from most relevant to least relevant using term frequency.

***Note***

Next code block requires user input.

Use the example ```who started the war between Minya and Thebe``` to get Hercules_Panhellenios_(Earth-616) [document id = 35] as the result document.

This function tries to find the document with the most number of terms withing give query. So you can use whole paragraphs from the document HTML and it should
return an exact match.

***Testing***

Copy and paste a string from a particular avenger's page as query. The function should return that avengers document url in results.

In [11]:
def complex_term(query):
    dic = {}
    posts = []
    max_post_ref = (0, 0) # holds a size, index reference for the post with maximum document ids
    get_doc_url = lambda lookup_id: [url for url, doc_id in docids_dic.items() if lookup_id == doc_id ][0]

    try:
        # remove punctuations, cast to lowercase and tokenise query
        tokens = tokeniser(query.lower().translate(str.maketrans('', '', string.punctuation)))
        # group all postings results from term searhes into a list
        for idx, term in enumerate(tokens):
            post = postings_dic.get(term)
            if post:
                doc_ids_size = len(post['doc_ids'])
                max_post_ref = (doc_ids_size, idx) if doc_ids_size > max_post_ref[0] else (max_post_ref[0], max_post_ref[1])
                posts.append(post)
        
        for i in range(max_post_ref[0]):
            freq = 0
            post_index = 0
            intersection_count = 0
            while post_index < len(posts):
                for k in range(len(posts[post_index]['doc_ids'])):
                    if posts[post_index]['doc_ids'][k] == posts[max_post_ref[1]]['doc_ids'][i]:
                        intersection_count += 1
                        freq += posts[post_index]['doc_freqs'][k]
                        break
                post_index += 1
            if intersection_count == len(posts):
                url = get_doc_url(posts[max_post_ref[1]]['doc_ids'][i])
                dic[url] = freq
    
        # sort the dictionary by frequency and print results
        ranked_result = [k for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)]

        if len(ranked_result):
            print(*ranked_result, sep='\n')
            # uncomment to get a print out of the sorted rankings map
            # print({k: v for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)})
        else:
            raise Exception()

    except Exception:
        print('No matches found!')

query = input('Enter any string as query.\n(eg. \n1. Peter Benjamin Parker was born in Queens) or \n2. Benjamin AND Parker AND born:')
complex_term(query)

Peter_Parker_(Earth-616)
Hercules_Panhellenios_(Earth-616)
Anthony_Stark_(Earth-616)
Bruce_Banner_(Earth-616)
James_Howlett_(Earth-616)
Eugene_Thompson_(Earth-616)
Shang-Chi_(Earth-616)
Stephen_Strange_(Earth-616)
Steven_Rogers_(Earth-616)
Reed_Richards_(Earth-616)
Brian_Braddock_(Earth-616)
Robert_Baldwin_(Earth-616)
Pietro_Maximoff_(Earth-616)
Dennis_Sykes_(Earth-616)
Natalia_Romanova_(Earth-616)
Jessica_Drew_(Earth-616)
Wanda_Maximoff_(Earth-616)
Miles_Morales_(Earth-1610)
Barbara_Morse_(Earth-616)
Venom_(Symbiote)_(Earth-616)
Eric_Brooks_(Earth-616)
Samuel_Alexander_(Earth-616)
Thaddeus_Ross_(Earth-616)
Otto_Octavius_(Earth-616)
Susan_Storm_(Earth-616)
Julia_Carpenter_(Earth-616)
Luke_Cage_(Earth-616)
Captain_Universe_(Earth-616)
James_Buchanan_Barnes_(Earth-616)
Ororo_Munroe_(Earth-616)
William_Baker_(Earth-616)
Nadia_Van_Dyne_(Earth-616)
Eric_Masterson_(Earth-616)
Samuel_Wilson_(Earth-616)
Janet_Van_Dyne_(Earth-616)
Clinton_Barton_(Earth-616)
Henry_McCoy_(Earth-616)
Maria_Hill_(E